In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import csv
import warnings

In [2]:
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', None)

In [3]:
def plot_columns(df):
    colors = plt.cm.tab20(range(df.shape[1]))  

    for i, column in enumerate(df.columns):
        color = colors[i % 24]

        plt.figure()
        plt.plot(df.index, df[column], color=color)
        plt.xlabel('Ano')
        plt.ylabel('Valor')
        plt.title(column)
        plt.xticks(rotation=45)

    plt.show()
    
    
def plot_linear_regression(column, linear_a, linear_b, linear_a_half, linear_b_half):
    plt.figure(figsize=(10, 6))
    plt.scatter(column.index, column, label='Original Values')
    plt.plot(column.index, linear_a * column.index + linear_b, color='red', label='Linear Regression')
    plt.plot(column.index, linear_a_half * column.index + linear_b_half, color='green', label='Recent Linear Regression')
    plt.xlabel('Index')
    plt.ylabel('Value')
    plt.legend()
    plt.title('Original Values vs. Linear Regression vs. Recent Linear Regression')
    plt.grid(True)
    plt.show()

In [4]:
def centered_moving_average(column, window_action, window_size):
    start_index, end_index = window_action
    data = column.to_numpy()

    data_length = len(data)

    for i in range(start_index, end_index + 1):
        if not np.isnan(data[i]):
            init = data[i]
            start = max(0, i - window_size)
            end = min(data_length, i + window_size + 1)
            window = data[start:end]
            valid_values = [value for value in window if not pd.isna(value)]
            if len(valid_values) > 0:
                data[i] = np.mean(valid_values)

    modified_column = pd.Series(data, name=column.name)

    return modified_column

In [5]:
# Polynomial Interpolation data inputation method
def four_ends_filler(column):
    if  pd.api.types.is_numeric_dtype(column):
        for i in range(4):
            column = first_filler(column, i, len(column) - 1 - i)
            column = last_filler(column, i, len(column) - 1 - i)
    return column

def four_ends_linear_filler(column):
    if  pd.api.types.is_numeric_dtype(column):
        size = len(column) - 1
        column = first_filler(column, 0, size)
        column = last_filler(column, 0, size)
        for i in range(1, 4):
            value = column.at[i]
            if (pd.isnull(value) or np.isnan(value) or (value == '')):
                start_first = i - 1
                end_first = next_valid_index(column, i)
                linear_a = (column.at[end_first] - column.at[i - 1])/(end_first - start_first)
                linear_b = column.at[start_first] - linear_a*start_first
                column.at[i] = linear_a*i + linear_b
            x = size - 4 + i
            value = column.at[x]
            if (pd.isnull(value) or np.isnan(value) or (value == '')):
                start_last = prev_valid_index(column, x)
                end_last = next_valid_index(column, x)
                gap = list(range(start_last + 1, end_last))
                linear_a = (column.at[end_last] - column.at[start_last])/(end_last - start_last)
                linear_b = column.at[start_last] - linear_a*start_last
                column.at[x] = linear_a*x + linear_b
    return column

def first_filler(column, init, end):
    if pd.isnull(column.at[init]) or np.isnan(column.at[init]):
        first_valid_index = 0
        while first_valid_index <= end and (pd.isnull(column.at[first_valid_index]) or 
                                             np.isnan(column.at[first_valid_index])):
            first_valid_index += 1
        if first_valid_index > end:
            column.at[init] = 0.0
        else:
            column.at[init] = column.at[first_valid_index]
    return column

def extract_valid_values(column):
    valid_values = []
    valid_indexes = []
    for index, value in enumerate(column):
        if not pd.isnull(value) and not np.isnan(value):
            valid_values.append(value)
            valid_indexes.append(index)
    extracted_column = pd.Series(
        valid_values,
        index=valid_indexes
    )
    return extracted_column

def linear_regression_first_four_filler(column):
    size = len(column) - 1
    processed_column = extract_valid_values(column)
    mean_valid_values_quarter = processed_column.mean()/4
    mean_valid_values_quarter_upper = processed_column.mean() + mean_valid_values_quarter
    
    model = sm.OLS(processed_column, sm.add_constant(processed_column.index))
    result = model.fit()
    linear_a = result.params[1]
    linear_b = result.params['const']
    half_column = processed_column[:(len(processed_column) // 2)]
    model2 = sm.OLS(half_column, sm.add_constant(half_column.index))
    result2 = model2.fit()
    linear_a_half = result2.params[1]
    linear_b_half = result2.params['const']
    
    for i in range(0,4):
        if pd.isnull(column.at[i]) or np.isnan(column.at[i]):
            projected = ((linear_a*i + linear_b) + (linear_a_half*i + linear_b_half))/2
            if projected < mean_valid_values_quarter:
                column.at[i] = mean_valid_values_quarter
            elif projected > mean_valid_values_quarter_upper:
                column.at[i] = mean_valid_values_quarter_upper
            else:
                column.at[i] = projected
        x = size - 4 + i
        if pd.isnull(column.at[x]) or np.isnan(column.at[x]):
            projected = ((linear_a*i + linear_b) + (linear_a_half*i + linear_b_half))/2
            if projected < mean_valid_values_quarter:
                column.at[x] = mean_valid_values_quarter
            elif projected > mean_valid_values_quarter_upper:
                column.at[x] = mean_valid_values_quarter_upper
            else:
                column.at[x] = projected
    plot_linear_regression(column, linear_a, linear_b, linear_a_half, linear_b_half)
    return column
    
def last_filler(column, init, end):
    if pd.isnull(column.at[end]) or np.isnan(column.at[end]):
        last_valid_index = len(column) -1
        while last_valid_index >= init and (pd.isnull(column.at[last_valid_index]) or 
                                         np.isnan(column.at[last_valid_index])):
            last_valid_index -= 1
        if last_valid_index < init:
            column.at[end] = 0.0
        else:
            column.at[end] = column.at[last_valid_index]
    return column
  
def neareast_four_indexes(column, start, end):
    return prev_four_valid_indexes(column, start) + next_four_valid_indexes(column, end)

def prev_four_valid_indexes(column, pos):
    prev_index = pos
    index_prev_list = [0]*4
    size = len(column) - 1
    if pos > 0 and pos < size:
        for i in range(4):
            index_prev_list[3 - i] = prev_index
            prev_index -= 1
            while prev_index >= 0 and (pd.isnull(column.at[prev_index]) or np.isnan(column.at[prev_index])):
                prev_index -= 1
    return index_prev_list  

def next_four_valid_indexes(column, pos):
    next_index = pos
    index_next_list = [0]*4
    size = len(column) - 1
    if pos > 0 and pos < size:
        for i in range(4):
            index_next_list[i] = next_index
            next_index += 1
            while next_index <= size and (pd.isnull(column.at[next_index]) or np.isnan(column.at[next_index])):
                next_index += 1
    return index_next_list  

def prev_valid_index(column, pos):
    prev_index = pos
    size = len(column) -1
    if pos >= 0 and pos < size:
        prev_index -= 1
        while prev_index >= 0 and (pd.isnull(column.at[prev_index]) or np.isnan(column.at[prev_index])):
            prev_index -= 1
    return prev_index  

def next_valid_index(column, pos):
    next_index = pos
    size = len(column) -1
    if pos >= 0 and pos < size:
        next_index += 1
        while next_index <= size and (pd.isnull(column.at[next_index]) or np.isnan(column.at[next_index])):
            next_index += 1
    return next_index

def interpolating_polynomial(nearest_indexes, nearest_values):
    coeffs = np.polyfit(nearest_indexes, nearest_values, 3)
    return np.poly1d(coeffs)

def polynomial_filler(column):
    size = len(column) -1
    if column.isnull().any() and column.isna().any():
        for i, value in enumerate(column):
            if (i > 3 and i < size - 3) and (pd.isnull(value) or np.isnan(value) or (value == '')):
                nearest_values = []
                start = prev_valid_index(column, i)
                end = next_valid_index(column, i)
                gap = list(range(start + 1, end))
                nearest_indexes = neareast_four_indexes(column, start, end)
                for index in nearest_indexes:
                    nearest_values.append(column.at[index])
                polynomial = interpolating_polynomial(nearest_indexes, nearest_values)
                for x in gap:
                    column.at[x] = polynomial(x)
                plot_polynomial(polynomial, nearest_indexes[0], nearest_indexes[len(nearest_indexes) - 1], 50, gap, nearest_indexes, column.name)
    return column

def plot_polynomial(polynomial, start, end, precision, interpolated_list, nearest_indexes, column_name):
    x = np.linspace(start, end, precision)
    y = polynomial(x)

    plt.figure()

    plt.plot(x, y)

    interpolated_x = interpolated_list  
    interpolated_y = polynomial(interpolated_x)
    plt.scatter(interpolated_x, interpolated_y, color='red')
    interpolated_x = nearest_indexes  
    interpolated_y = polynomial(interpolated_x)
    plt.scatter(interpolated_x, interpolated_y, color='green')
    
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title(column_name)

    plt.show()
    
def replace_negative_with_float_zeros(x):
    return 0.0 if x < 0 else x

In [6]:
def read_csv_replace_missing_by_polynomial(file_path, skiped_columns = 0, skiped_rows = 0):
    with open(file_path, 'r') as f:
        
        df = pd.read_csv(file_path, skiprows=skiped_rows)

        i = 0
        for col in df.columns: # Elimina missing values por médias e transforma em float
            if i > skiped_columns:
                df[col] = df[col].replace('-9999', np.nan)
                df[col] = df[col].replace('-', np.nan)
                df[col] = df[col].replace('', np.nan).astype(float)
                df[col] = four_ends_filler(df[col])
                df[col] = centered_moving_average(df[col], (0, len(df[col]) - 1), 24) #2 anos
                df[col] = polynomial_filler(df[col]) 
            i += 1
        return df

In [11]:
df = pd.read_csv('custo_m2.csv')
df.set_index('UF', inplace=True)
df = df.T
df

UF,Rondônia,Acre,Amazonas,Roraima,Pará,Amapá,Tocantins,Maranhão,Piauí,Ceará,...,Espírito Santo,Rio de Janeiro,São Paulo,Paraná,Santa Catarina,Rio Grande do Sul,Mato Grosso do Sul,Mato Grosso,Goiás,Distrito Federal
2003,8797.19,9092.37,10252.87,11474.84,9342.25,9537.36,9892.63,9442.87,8322.00,9178.47,...,8925.60,10586.09,10992.91,10701.75,9863.32,10010.23,9209.47,9509.38,9403.03,10768.04
2003.1,8996.67,9212.65,10320.95,11556.21,9506.07,9647.72,10023.15,9592.82,8552.80,9305.17,...,9048.34,10733.35,11100.76,10753.39,9944.57,10163.51,9325.55,9567.69,9478.93,10912.73
2003.2,9030.02,9256.72,10422.98,11635.38,9670.32,9744.17,10145.48,9699.91,8608.56,9379.04,...,9126.75,10862.11,11216.85,10785.98,9990.99,10213.43,9433.85,9583.99,9548.63,10962.66
2003.3,9148.10,9433.16,10486.95,11695.00,9703.79,9822.52,10193.95,9816.51,8741.33,9439.22,...,9222.02,11582.85,11337.91,10872.89,10175.92,10342.42,9506.08,9694.28,9616.78,11031.47
2003.4,9149.99,9892.15,10509.97,11902.96,9802.50,9892.03,10352.16,9850.66,8867.82,9773.28,...,9396.88,11666.46,11906.68,10936.06,10242.62,10400.49,9606.89,9744.49,9696.31,11729.34
2003.5,9627.14,9913.09,10543.52,12006.94,9815.82,10085.44,10436.85,9939.44,8897.82,9764.57,...,9843.53,11783.46,12058.24,10979.36,10916.20,10662.01,10170.91,9771.45,10296.90,11849.62
2003.6,9642.04,10001.86,11008.12,12069.28,9854.44,10129.76,11015.48,9954.21,8917.71,9810.46,...,9889.77,11823.99,12133.67,11583.88,10977.88,10742.42,10226.08,9798.54,10298.70,11927.46
2003.7,9686.75,10035.05,11035.81,12078.26,9861.53,10198.17,11069.51,10227.41,9034.45,9885.50,...,9916.44,12069.95,12365.08,11628.10,11072.18,10798.76,10281.93,9846.52,10322.63,11962.79
2003.8,9777.57,10066.47,11069.56,12498.64,9877.57,10205.00,11079.02,10275.61,9045.62,9910.14,...,10062.99,12177.41,12410.88,11640.64,11103.94,10852.55,10331.62,9991.13,10401.08,12039.44
2003.9,9809.77,10134.67,11090.73,12507.85,10162.36,10261.60,11097.38,10378.38,9088.29,9915.63,...,10088.93,12225.48,12466.40,11646.56,11126.47,11147.73,10340.96,10268.01,10458.93,12061.09


In [14]:
def add_suffix_to_columns(df, suffix):
    df.columns = [col + suffix for col in df.columns]

add_suffix_to_columns(df, "- custo m2 (R$)")

In [15]:
import re

def correct_index(df):
    new_index = []
    for i, index_value in enumerate(df.index):
        corrected_index = re.sub(r'\.\d{1,2}', '', index_value)
        new_index.append(f"{corrected_index}-{i % 12 + 1}")
    df.index = new_index
    return df

df = correct_index(df)
df

,Rondônia- custo m2 (R$),Acre- custo m2 (R$),Amazonas- custo m2 (R$),Roraima- custo m2 (R$),Pará- custo m2 (R$),Amapá- custo m2 (R$),Tocantins- custo m2 (R$),Maranhão- custo m2 (R$),Piauí- custo m2 (R$),Ceará- custo m2 (R$),...,Espírito Santo- custo m2 (R$),Rio de Janeiro- custo m2 (R$),São Paulo- custo m2 (R$),Paraná- custo m2 (R$),Santa Catarina- custo m2 (R$),Rio Grande do Sul- custo m2 (R$),Mato Grosso do Sul- custo m2 (R$),Mato Grosso- custo m2 (R$),Goiás- custo m2 (R$),Distrito Federal- custo m2 (R$)
2003-1-1,8797.19,9092.37,10252.87,11474.84,9342.25,9537.36,9892.63,9442.87,8322.00,9178.47,...,8925.60,10586.09,10992.91,10701.75,9863.32,10010.23,9209.47,9509.38,9403.03,10768.04
2003-2-2,8996.67,9212.65,10320.95,11556.21,9506.07,9647.72,10023.15,9592.82,8552.80,9305.17,...,9048.34,10733.35,11100.76,10753.39,9944.57,10163.51,9325.55,9567.69,9478.93,10912.73
2003-3-3,9030.02,9256.72,10422.98,11635.38,9670.32,9744.17,10145.48,9699.91,8608.56,9379.04,...,9126.75,10862.11,11216.85,10785.98,9990.99,10213.43,9433.85,9583.99,9548.63,10962.66
2003-4-4,9148.10,9433.16,10486.95,11695.00,9703.79,9822.52,10193.95,9816.51,8741.33,9439.22,...,9222.02,11582.85,11337.91,10872.89,10175.92,10342.42,9506.08,9694.28,9616.78,11031.47
2003-5-5,9149.99,9892.15,10509.97,11902.96,9802.50,9892.03,10352.16,9850.66,8867.82,9773.28,...,9396.88,11666.46,11906.68,10936.06,10242.62,10400.49,9606.89,9744.49,9696.31,11729.34
2003-6-6,9627.14,9913.09,10543.52,12006.94,9815.82,10085.44,10436.85,9939.44,8897.82,9764.57,...,9843.53,11783.46,12058.24,10979.36,10916.20,10662.01,10170.91,9771.45,10296.90,11849.62
2003-7-7,9642.04,10001.86,11008.12,12069.28,9854.44,10129.76,11015.48,9954.21,8917.71,9810.46,...,9889.77,11823.99,12133.67,11583.88,10977.88,10742.42,10226.08,9798.54,10298.70,11927.46
2003-8-8,9686.75,10035.05,11035.81,12078.26,9861.53,10198.17,11069.51,10227.41,9034.45,9885.50,...,9916.44,12069.95,12365.08,11628.10,11072.18,10798.76,10281.93,9846.52,10322.63,11962.79
2003-9-9,9777.57,10066.47,11069.56,12498.64,9877.57,10205.00,11079.02,10275.61,9045.62,9910.14,...,10062.99,12177.41,12410.88,11640.64,11103.94,10852.55,10331.62,9991.13,10401.08,12039.44
2003-10-10,9809.77,10134.67,11090.73,12507.85,10162.36,10261.60,11097.38,10378.38,9088.29,9915.63,...,10088.93,12225.48,12466.40,11646.56,11126.47,11147.73,10340.96,10268.01,10458.93,12061.09


In [16]:
for column in df.columns:
    file_name = f"2003_raw_mo_{column}.csv"
    column_df = df[[column]]
    column_df.to_csv(file_name, index=True)

In [9]:
# df.to_csv('custo_m2_ajustado.csv', index=True)

In [10]:
df = read_csv_replace_missing_by_polynomial('custo_m2_ajustado.csv')
df.set_index('Ano', inplace=True)
df

,Rondônia,Acre,Amazonas,Roraima,Pará,Amapá,Tocantins,Maranhão,Piauí,Ceará,...,Espírito Santo,Rio de Janeiro,São Paulo,Paraná,Santa Catarina,Rio Grande do Sul,Mato Grosso do Sul,Mato Grosso,Goiás,Distrito Federal
Ano,,,,,,,,,,,,,,,,,,,,,
2003-1,10006.623600,10254.061200,11283.694800,12652.753600,10387.404000,10518.903600,11275.335200,10548.340000,9410.948400,10216.279200,...,10264.686400,12454.864800,12606.426400,11767.370000,11270.272400,11314.228000,10575.832000,10522.560000,10601.325200,12173.796800
2003-2,10093.568215,10335.111969,11377.134800,12757.336292,10470.519385,10623.124754,11378.766738,10647.107692,9501.490708,10302.959969,...,10362.759477,12574.990185,12718.304862,11852.421538,11377.902015,11415.287231,10674.618538,10614.965000,10688.670969,12272.096031
2003-3,10174.904141,10411.052710,11466.977022,12860.789996,10546.359385,10723.388458,11476.693775,10736.259174,9582.303300,10382.029969,...,10458.009107,12687.426111,12824.573380,11934.318946,11480.211275,11510.305379,10766.098168,10702.843148,10770.309117,12362.351216
2003-4,10253.662356,10486.231639,11551.599522,12962.516425,10614.915813,10817.834529,11568.023775,10818.881674,9659.062943,10456.629969,...,10551.050535,12814.530039,12924.495880,12015.342874,11578.298418,11603.969307,10853.134597,10785.405648,10847.723760,12449.345859
2003-5,10327.460287,10566.989914,11632.378833,13063.753666,10682.772020,10909.780391,11656.641017,10897.469605,9730.815357,10527.718245,...,10642.401914,12915.463832,13031.542087,12093.451150,11673.320486,11692.135859,10952.713907,10863.369096,10930.821691,12543.454480
2003-6,10411.174287,10631.423914,11713.338499,13159.324666,10745.783353,10995.670391,11738.354684,10974.782605,9797.367024,10591.308245,...,10734.939581,13011.255499,13117.532087,12180.915150,11764.486153,11777.788526,11045.489574,10938.122430,11009.628691,12612.328146
2003-7,10477.421383,10694.321011,11802.026241,13248.373053,10807.415289,11073.322327,11815.514361,11047.643895,9861.661862,10654.736632,...,10810.037646,13102.741950,13197.411442,12264.968376,11832.479379,11857.726268,11116.889896,11017.884688,11067.567078,12676.302340
2003-8,10541.390133,10753.133824,11873.959679,13334.437428,10866.937164,11148.308889,11884.442173,11119.600145,9922.931550,10715.543195,...,10882.506708,13191.451013,13274.391754,12328.331189,11898.219066,11933.157205,11184.972084,11094.937813,11124.383641,12736.728277
2003-9,10602.560133,10810.976248,11944.116042,13418.329852,10924.604740,11219.100404,11949.800961,11182.441054,9978.644580,10772.681983,...,10953.063375,13270.282528,13342.029330,12387.795431,11959.815430,12005.503569,11249.866023,11168.603267,11179.665156,12794.057368


In [11]:
df_mo_2003 = df
for column in df.columns:
    file_name = f"2003_mo_{column}.csv"
    column_df = df[[column]]
    column_df.to_csv(file_name, index=True)